In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from src import tools
import glob
import nibabel as nib
from nilearn import plotting

/Applications/anaconda3/envs/SIfMRI_analysis/lib/python3.6/site-packages/nilearn/__init__.py:69: FutureWarning: Python 3.6 support is deprecated and will be removed in release 0.10 of Nilearn. Consider switching to Python 3.8 or 3.9.
  _python_deprecation_warnings()


In [2]:
top_dir = '/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis'
data_dir = f'{top_dir}/data/raw'
out_dir = f'{top_dir}/data/interm'
figure_dir = f'{top_dir}/reports/figures'

In [3]:
# Get the feature names for the annotated model
 features = pd.read_csv(f'{data_dir}/annotations/annotations.csv').columns.to_list()
features.remove('video_name')

In [4]:
n_subjs = 4
n_features = len(features)
n_conditions = 200

In [5]:
mask = np.load(f'{out_dir}/group_reliability/sub-all_reliability-mask.npy')
mask_im = nib.load(f'{out_dir}/group_reliability/sub-all_stat-rho_statmap.nii.gz')
n_voxels = sum(mask)

In [6]:
yhat = np.zeros((n_features, n_conditions, n_voxels))
y = np.zeros((n_conditions, n_voxels))
for sid_ in range(n_subjs):
    sid = str(sid_+1).zfill(2)
    yhat += np.load(f'{out_dir}/voxelwise_encoding/sub-{sid}_yhat.npy')
    y += np.load(f'{out_dir}/voxelwise_encoding/sub-{sid}_y.npy')
yhat /= n_subjs
y /= n_subjs

In [31]:
def mkbrain(mask, arr, im):
    out_im = np.zeros(mask.size)
    inds = np.where(mask)[0]
    out_im[inds] = arr
    return nib.Nifti1Image(out_im.reshape(im.shape), affine=im.affine)

In [32]:
rs = np.zeros((n_features, y.shape[-1]))
for fi, feature in enumerate(features):
    rs = tools.corr2d(y, yhat[fi,...])
    rs_im = mkbrain(mask, rs, mask_im)
    outname = f'{figure_dir}/voxelwise_encoding/{feature}.pdf'
    plotting.plot_img_on_surf(rs_im, inflate=True,
                              threshold=0.01,
                              output_file=outname,
                              views=['lateral', 'medial', 'ventral'])

/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/src/tools.py:11: RuntimeWarning: invalid value encountered in true_divide
  r = np.sum((x_m * y_m), axis=0) / np.sqrt(np.sum((x_m * x_m), axis=0) * np.sum((y_m * y_m), axis=0))
/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/src/tools.py:11: RuntimeWarning: invalid value encountered in true_divide
  r = np.sum((x_m * y_m), axis=0) / np.sqrt(np.sum((x_m * x_m), axis=0) * np.sum((y_m * y_m), axis=0))
/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/src/tools.py:11: RuntimeWarning: invalid value encountered in true_divide
  r = np.sum((x_m * y_m), axis=0) / np.sqrt(np.sum((x_m * x_m), axis=0) * np.sum((y_m * y_m), axis=0))
/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/src/tools.py:11: RuntimeWarning: invalid value encountered in true_divide
  r = np.sum((x_m * y_m), axis=0) / np.sqrt(np.sum((x_m * x_m), axis=0) * np.sum((y_m * y_m), axis=0))
/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/src/too